# Getting data into Python

Outline:

- ASCII Files: numpy.loadtxt, astropy.io.ascii, read_csv (pandas)
- Reading/Writing FITS files: astropy.io.fits, fitsio
- IDL .sav files: scipy.readsav
- Pandas

# CSV data

In [1]:
import os
import numpy as np
import requests

In [2]:
# get some CSV data from the SDSS SQL server
URL = "http://skyserver.sdss.org/dr12/en/tools/search/x_sql.aspx"

cmd = """
SELECT TOP 1000
    p.u, p.g, p.r, p.i, p.z, s.class, s.z, s.zerr
FROM
    PhotoObj AS p
JOIN
    SpecObj AS s ON s.bestobjid = p.objid
WHERE
    p.u BETWEEN 0 AND 19.6 AND
    p.g BETWEEN 0 AND 20 AND
    s.class = 'GALAXY'
"""
if not os.path.exists('galaxy_colors.csv'):
    cmd = ' '.join(map(lambda x: x.strip(), cmd.split('\n')))
    response = requests.get(URL, params={'cmd': cmd, 'format':'csv'})
    with open('galaxy_colors.csv', 'w') as f:
        f.write(response.text)

In [3]:
!ls -lh galaxy_colors.csv

-rw-rw-r-- 1 kyle kyle 74K Sep 24 15:54 galaxy_colors.csv


In [4]:
!more galaxy_colors.csv

#Table1
u,g,r,i,z,class,z1,zerr
19.41061,18.23754,17.58132,17.20153,16.90159,GALAXY,0.03212454,6.06623E-06
19.54964,17.95799,17.02898,16.531,16.13408,GALAXY,0.1213151,2.358919E-05
18.74425,17.37778,16.80538,16.51149,16.28756,GALAXY,0.04876465,1.378529E-05
17.55033,15.75007,15.02809,14.66306,14.34982,GALAXY,0.04028672,1.167005E-05
17.60645,16.16628,15.51308,15.15529,14.87411,GALAXY,0.0254747,1.205017E-05
19.46927,18.18101,17.59062,17.25874,16.94567,GALAXY,0.03616738,8.249292E-06
19.58999,18.23981,17.54194,17.17573,16.92423,GALAXY,0.07254888,1.603681E-05
18.52309,16.65203,15.9179,15.47603,15.16455,GALAXY,0.06675781,1.785021E-05
18.7319,17.42271,16.80514,16.47006,16.18039,GALAXY,0.03646222,1.014089E-05
19.51618,18.32554,17.63795,17.25495,17.02355,GALAXY,0.1380212,9.179801E-06
18.97668,17.5344,17.04177,16.80851,16.6608,GALAXY,0.03877712,1.343119E-05
18.28252,16.52093,15.60923,15.08733,14.62764,GALAXY,0.0406868,9.147252E-06
18.79809,17.15676,16.46291,16.05119,15.75717,GALAXY,0.03758542,1.21

## Using numpy.loadtxt

In [5]:
dtype=[('u', 'f8'),
       ('g', 'f8'),
       ('r', 'f8'),
       ('i', 'f8'),
       ('z', 'f8'),
       ('class', 'S10'),
       ('redshift', 'f8'),
       ('redshift_err', 'f8')]
data = np.loadtxt('galaxy_colors.csv', skiprows=2, delimiter=',', dtype=dtype)

In [6]:
data[:10]

array([ (19.41061, 18.23754, 17.58132, 17.20153, 16.90159, b'GALAXY', 0.03212454, 6.06623e-06),
       (19.54964, 17.95799, 17.02898, 16.531, 16.13408, b'GALAXY', 0.1213151, 2.358919e-05),
       (18.74425, 17.37778, 16.80538, 16.51149, 16.28756, b'GALAXY', 0.04876465, 1.378529e-05),
       (17.55033, 15.75007, 15.02809, 14.66306, 14.34982, b'GALAXY', 0.04028672, 1.167005e-05),
       (17.60645, 16.16628, 15.51308, 15.15529, 14.87411, b'GALAXY', 0.0254747, 1.205017e-05),
       (19.46927, 18.18101, 17.59062, 17.25874, 16.94567, b'GALAXY', 0.03616738, 8.249292e-06),
       (19.58999, 18.23981, 17.54194, 17.17573, 16.92423, b'GALAXY', 0.07254888, 1.603681e-05),
       (18.52309, 16.65203, 15.9179, 15.47603, 15.16455, b'GALAXY', 0.06675781, 1.785021e-05),
       (18.7319, 17.42271, 16.80514, 16.47006, 16.18039, b'GALAXY', 0.03646222, 1.014089e-05),
       (19.51618, 18.32554, 17.63795, 17.25495, 17.02355, b'GALAXY', 0.1380212, 9.179801e-06)], 
      dtype=[('u', '<f8'), ('g', '<f8'), ('r'

## Using astropy.io.ascii

In [8]:
from astropy.io import ascii

/home/kyle/.conda/lib/python3.4/site-packages/IPython/kernel/__init__.py:13: ShimWarning: The `IPython.kernel` package has been deprecated. You should import from ipykernel or jupyter_client instead.
  "You should import from ipykernel or jupyter_client instead.", ShimWarning)


In [9]:
data = ascii.read('galaxy_colors.csv', format='csv', comment='#')

In [10]:
type(data)

astropy.table.table.Table

In [11]:
data[:10]

u,g,r,i,z,class,z1,zerr
float64,float64,float64,float64,float64,str192,float64,float64
19.41061,18.23754,17.58132,17.20153,16.90159,GALAXY,0.03212454,6.06623e-06
19.54964,17.95799,17.02898,16.531,16.13408,GALAXY,0.1213151,2.358919e-05
18.74425,17.37778,16.80538,16.51149,16.28756,GALAXY,0.04876465,1.378529e-05
17.55033,15.75007,15.02809,14.66306,14.34982,GALAXY,0.04028672,1.167005e-05
17.60645,16.16628,15.51308,15.15529,14.87411,GALAXY,0.0254747,1.205017e-05
19.46927,18.18101,17.59062,17.25874,16.94567,GALAXY,0.03616738,8.249292e-06
19.58999,18.23981,17.54194,17.17573,16.92423,GALAXY,0.07254888,1.603681e-05
18.52309,16.65203,15.9179,15.47603,15.16455,GALAXY,0.06675781,1.785021e-05
18.7319,17.42271,16.80514,16.47006,16.18039,GALAXY,0.03646222,1.014089e-05


## Using pandas

In [12]:
import pandas

In [13]:
data = pandas.read_csv('galaxy_colors.csv', comment='#')

In [14]:
type(data)

pandas.core.frame.DataFrame

In [15]:
data.head()

,u,g,r,i,z,class,z1,zerr
0,19.41061,18.23754,17.58132,17.20153,16.90159,GALAXY,0.032125,0.000006
1,19.54964,17.95799,17.02898,16.53100,16.13408,GALAXY,0.121315,0.000024
2,18.74425,17.37778,16.80538,16.51149,16.28756,GALAXY,0.048765,0.000014
3,17.55033,15.75007,15.02809,14.66306,14.34982,GALAXY,0.040287,0.000012
4,17.60645,16.16628,15.51308,15.15529,14.87411,GALAXY,0.025475,0.000012


In [16]:
data.describe()

,u,g,r,i,z,z1,zerr
count,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000,1000.000000
mean,18.847712,17.362790,16.652535,16.277254,16.015587,0.079304,0.000014
std,0.705162,0.777843,0.902157,0.942336,0.988064,0.051818,0.000011
min,14.065570,12.779670,12.145150,11.757920,11.650580,0.000000,0.000000
25%,18.537655,16.973130,16.168653,15.765792,15.432625,0.050569,0.000008
50%,19.043970,17.492975,16.719680,16.321100,16.040580,0.077798,0.000012
75%,19.355695,17.924820,17.313200,16.976025,16.745777,0.093881,0.000017
max,19.599320,19.260320,24.802040,24.361810,22.826910,0.760918,0.000233


In [ ]:
# Pandas reads from *lots* of different data sources
pandas.read_

# Specialized text formats

In [17]:
# get some data from CDS
prefix = "http://cdsarc.u-strasbg.fr/vizier/ftp/cats/J/ApJ/686/749/"
for fname in ["ReadMe", "table10.dat"]:
    if not os.path.exists(fname):
        response = requests.get(prefix + fname)
        with open(fname, 'w') as f:
            f.write(response.text)

In [ ]:
!cat table10.dat

In [ ]:
!cat ReadMe

In [20]:
# must specify the "readme" here.
data = ascii.read("table10.dat", format='cds', readme="ReadMe")

In [21]:
data

SN,JD,Tel,Bmag,e_Bmag,Vmag,e_Vmag,Rmag,e_Rmag,Imag,e_Imag
,d,,mag,mag,mag,mag,mag,mag,mag,mag
str288,float64,str448,float64,float64,float64,float64,float64,float64,float64,float64
SN 1999aa,2451221.81,LICK 1m DEWAR2,15.828,0.032,--,--,--,--,--,--
SN 1999aa,2451222.67,YALO,15.642,0.018,15.68,0.028,15.689,0.04,15.717,0.105
SN 1999aa,2451223.67,YALO,15.462,0.02,--,--,15.486,0.036,15.514,0.083
SN 1999aa,2451225.65,YALO,15.211,0.017,15.26,0.028,15.276,0.03,15.312,0.025
SN 1999aa,2451227.73,LICK 1m DEWAR2,15.006,0.016,15.06,0.024,15.08,0.018,--,--
SN 1999aa,2451229.62,YALO,14.924,0.017,14.965,0.026,15.092,0.03,--,--
SN 1999aa,2451232.61,YALO,14.908,0.017,14.913,0.028,15.062,0.03,15.253,0.025
SN 1999aa,2451235.6,YALO,14.919,0.021,14.898,0.032,15.037,0.031,15.307,0.029


See http://astropy.readthedocs.org/en/stable/io/ascii/index.html for all the supported formats.

# Reading FITS files

Two options: `astropy.io.fits` (formerly pyfits) and `fitsio`.

In [ ]:
# get an SDSS image (can search for images from http://dr12.sdss3.org/fields/)
if not os.path.exists("frame-g-006728-4-0121.fits.bz2"):
    !wget http://dr12.sdss3.org/sas/dr12/boss/photoObj/frames/301/6728/4/frame-g-006728-4-0121.fits.bz2
if not os.path.exists("frame-g-006728-4-0121.fits"):
    !bunzip2 frame-g-006728-4-0121.fits.bz2

## astropy.io.fits

In [22]:
from astropy.io import fits

hdulist = fits.open("frame-g-006728-4-0121.fits")

In [23]:
hdulist

In [24]:
hdulist.info()

Filename: frame-g-006728-4-0121.fits
No.    Name         Type      Cards   Dimensions   Format
0    PRIMARY     PrimaryHDU      96   (2048, 1489)   float32   
1                ImageHDU         6   (2048,)      float32   
2                BinTableHDU     27   1R x 3C      [49152E, 2048E, 1489E]   
3                BinTableHDU     79   1R x 31C     [J, 3A, J, A, D, D, 2J, J, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, D, E, E]   


In [25]:
hdulist[0].data

array([[ 0.00395966,  0.0116272 ,  0.02697754, ..., -0.00365067,
         0.01544189,  0.0116272 ],
       [-0.01139832, -0.00756073, -0.00756073, ...,  0.00016785,
         0.0116272 ,  0.0269165 ],
       [ 0.01547241, -0.00756073, -0.01522827, ...,  0.01544189,
         0.00016856,  0.01925659],
       ..., 
       [ 0.00098419,  0.00866699,  0.00482178, ...,  0.03155518,
         0.00483704, -0.01806641],
       [-0.02206421, -0.02972412,  0.00866699, ...,  0.01629639,
         0.01629639,  0.01248169],
       [-0.02206421, -0.03356934,  0.01249695, ...,  0.00865173,
         0.00865173,  0.02011108]], dtype=float32)

In [ ]:
hdulist[0].header

## fitsio

(`pip install --no-deps fitsio`)

- Faster (mainly for tables)
- Does a better job with ASCII table extensions

In [27]:
import fitsio

In [28]:
f = fitsio.FITS("frame-g-006728-4-0121.fits")

In [29]:
# summary of file HDUs
f


  file: frame-g-006728-4-0121.fits
  mode: READONLY
  extnum hdutype         hduname[v]
  0      IMAGE_HDU       
  1      IMAGE_HDU       
  2      BINARY_TBL      
  3      BINARY_TBL      

In [30]:
# summary of first HDU
f[0]


  file: frame-g-006728-4-0121.fits
  extension: 0
  type: IMAGE_HDU
  image info:
    data type: f4
    dims: [1489,2048]

In [31]:
# Summary of 3rd HDU
f[2]


  file: frame-g-006728-4-0121.fits
  extension: 2
  type: BINARY_TBL
  rows: 1
  column info:
    ALLSKY              f4  array[256,192]
    XINTERP             f4  array[2048]
    YINTERP             f4  array[1489]

In [32]:
# Actually read the data.
data = f[0].read()
data

array([[ 0.00395966,  0.0116272 ,  0.02697754, ..., -0.00365067,
         0.01544189,  0.0116272 ],
       [-0.01139832, -0.00756073, -0.00756073, ...,  0.00016785,
         0.0116272 ,  0.0269165 ],
       [ 0.01547241, -0.00756073, -0.01522827, ...,  0.01544189,
         0.00016856,  0.01925659],
       ..., 
       [ 0.00098419,  0.00866699,  0.00482178, ...,  0.03155518,
         0.00483704, -0.01806641],
       [-0.02206421, -0.02972412,  0.00866699, ...,  0.01629639,
         0.01629639,  0.01248169],
       [-0.02206421, -0.03356934,  0.01249695, ...,  0.00865173,
         0.00865173,  0.02011108]], dtype=float32)

# Salvaging data from IDL

`scipy.io.readsav`: Formerly a separate `idlsave` module by Tom Robitaille.

In [34]:
from scipy.io import readsav

In [35]:
# Note: won't work unless you have this sav file!
data = readsav("150623434_det8_8100keV.sav")

In [36]:
data

{'events': array([  5.96046448e-08,   4.04059887e-04,   1.60408020e-03, ...,
          2.36982572e+03,   2.36982581e+03,   2.36982593e+03]),
 'tstart': 456747888.71404397}

In [37]:
len(data.events)

6653156

# Clean up downloaded files

In [ ]:
!rm galaxy_colors.csv
!rm ReadMe
!rm table10.dat
!rm frame-g-006728-4-0121.fits.bz2
!rm frame-g-006728-4-0121.fits